In [ ]:
'''
TO DO LIST:
-> Increase mutation extent considerably
-> Check maximum speed. IS BUGGED!! Solution? Maybe implement acceleration based control
-> Find optimal seperation and world sizes
-> Chunking for better performance in bigger worlds
-> Multicored Chunking
-> Ageing?
-> Show more advanced stats
'''

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import cv2 as cv
import sys
import time

from IPython.display import clear_output
%matplotlib inline

cv.useOptimized()

True

In [2]:
loc = "E:\Python work\Evolution"
os.chdir(loc) 
if os.getcwd() == loc:
    print("Successfully set directory!")
else:
    print("Error in setting directory. CWD is:", os.getcwd())
    
GEN = np.random.default_rng(seed = None)

Successfully set directory!


In [3]:
def activation(x):
    return np.arctan(x) * (2 / math.pi) #(2 / (1 + np.exp(-x))) - 1

def raycast(A, B, C, R):
    #0.38 MHz speed
    AB = math.dist(A, B)
    AC = math.dist(A, C)
    if AC > (AB + R):
        return False
    else:
        BC = math.dist(B, C)
        if BC > (AB + R):
            return False
        else:
            dist = (((B[0] - A[0]) * (A[1] - C[1])) - ((A[0] - C[0]) * (B[1] - A[1]))) / AB
            if abs(dist) > R:
                return False
            else:
                return True

def RotVect(theta):
    # Return rotation matrix - clockwise by theta
    theta *= (math.pi / 180)
    c = math.cos(theta)
    s = math.sin(theta)
    return np.array([[c, -s], [s, c]], dtype = np.float32)
        
def repr_chance(nrg):
    # Returns chance of reproduction per tick for this energy
    rc = (base_rep * (nrg ** 4)) * delta
    return rc


In [4]:
class Neural_Net:
    def __init__(self, n1, n2, n3, n4, scale, NetObj = None):
        if isinstance(NetObj, Neural_Net):
            self.n_inp = NetObj.n_inp
            self.n_hl1 = NetObj.n_hl1
            self.n_hl2 = NetObj.n_hl2
            self.n_outp = NetObj.n_outp
            self.total_nodes = NetObj.n_hl1 + NetObj.n_hl2 + NetObj.n_outp
            self.HL1 = NetObj.HL1
            self.HL1C = NetObj.HL1C
            self.HL2 = NetObj.HL2
            self.HL2C = NetObj.HL2C
            self.OP = NetObj.OP
            self.OPC = NetObj.OPC
        else:
            self.n_inp = n1
            self.n_hl1 = n2
            self.n_hl2 = n3
            self.n_outp = n4
            self.total_nodes = n2 + n3 + n4
            self.HL1 = scale * GEN.standard_normal(size = (n2, n1), dtype = np.float32)
            self.HL1C = scale * GEN.standard_normal(size = n2, dtype = np.float32)
            self.HL2 = scale * GEN.standard_normal(size = (n3, n2), dtype = np.float32)
            self.HL2C = scale * GEN.standard_normal(size = n3, dtype = np.float32)
            self.OP = scale * GEN.standard_normal(size = (n4, n3), dtype = np.float32)
            self.OPC = scale * GEN.standard_normal(size = n4, dtype = np.float32)
        
    def evaluate(self, input_vector):
        outp1 = activation((self.HL1 @ input_vector) + self.HL1C)
        outp2 = activation((self.HL2 @ outp1) + self.HL2C)
        outp = activation((self.OP @ outp2) + self.OPC)
        del outp1, outp2
        return outp

    def mutate(self, extent):
        self.HL1 += extent * GEN.standard_normal(size = (self.n_hl1, self.n_inp), dtype = np.float32)
        self.HL1C += extent * GEN.standard_normal(size = self.n_hl1, dtype = np.float32)
        self.HL2 += extent * GEN.standard_normal(size = (self.n_hl2, self.n_hl1), dtype = np.float32)
        self.HL2C += extent * GEN.standard_normal(size = self.n_hl2, dtype = np.float32)
        self.OP += extent * GEN.standard_normal(size = (self.n_outp, self.n_hl2), dtype = np.float32)
        self.OPC += extent * GEN.standard_normal(size = self.n_outp, dtype = np.float32)
               

In [5]:
MAX_ANGSPD = 60.0    # Max angular speed

class Pred_Stats:
    def __init__(self):
        self.kills = 0
        self.deaths = 0
        self.reprs = 0
    def pred_died(self):
        self.deaths += 1
    def pred_repr(self):
        self.reprs += 1
    def pred_killed(self):
        self.kills += 1
    
class Prey_Stats:
    def __init__(self):
        self.deaths = 0
        self.reprs = 0
    def prey_died(self):
        self.deaths += 1
    def prey_repr(self):
        self.reprs += 1

class Being:   
    max_ang_speed = MAX_ANGSPD  # Max angular speed
    max_energy = 50.0           # Energy Capacity
    pred_repr_mod = 0.5         # Predator reproduction chance modifier
    
    def __init__(self, X, Y, n1, n2, n3, scale, BeingObj = None):
        if BeingObj is not None:
            self.NNet = Neural_Net(n1, n2, n3, 2, scale, BeingObj.NNet)
            self.NNet.mutate(self.m_extent)
            self.location = BeingObj.location.copy()
            self.energy = self.max_energy * 0.6
        else:     
            self.NNet = Neural_Net(n1, n2, n3, 2, scale)
            self.location = np.array([X, Y], dtype = np.float32)
        self.spd = 0.0
        self.ang_speed = 0.0
        self.energy = 0.0
        self.look_direction = 0.0
        self.energy = self.max_energy
    
    def check_repr(self, isP = False):
        rchance = repr_chance((self.energy / self.max_energy))
        if isP == True:
            rchance = rchance * self.pred_repr_mod
        ran = np.random.rand(1)
        if ran <= rchance:
            return True
        else:
            return False

    def get_look(self):
        effdeg = (self.look_direction * math.pi) / 180.0
        lookX = math.sin(effdeg)
        lookY = math.cos(effdeg)
        del effdeg
        return [lookX, lookY]
    
    def move(self, dt):
        look = self.get_look()

        self.location[0] = (self.location[0] + (look[0] * self.spd * dt)) % BOUNDS
        self.location[1] = (self.location[1] + (look[1] * self.spd * dt)) % BOUNDS
        del look
    
##############################

class Prey(Being):   
    max_speed = 2.0             # units / sec  
    look_lines =  12            # Number of look lines - must be equal to n1
    sight_range = 4.0           # Range of sight lines
    size = 0.5                  # Size of object
    m_extent = 5.0              # Extent of mutation
    energy_drain = 2.0          # Energy drain per second
    n2 = 12                     # HL1 size
    n3 = 6                      # HL2 size
    energy_gain = 4.0           # Energy gain per second when still
    FoV = 360.0                 # Full FoV
    slow_threshold = 0.1        # Under what times of max speed do you gain energy
    repr_energy_loss = 18.0     # Energy lost when reproducing
    
    def __init__(self, x, y, scale, PreyObj = None):
        if isinstance(PreyObj, Prey):
            super(Prey, self).__init__(x, y, self.look_lines, self.n2, self.n3, scale, PreyObj)        
        else:
            super(Prey, self).__init__(x, y, self.look_lines, self.n2, self.n3, scale)
        self.isKilled = False
        
    def act(self, dt):
        if self.energy <= 0.0 or self.isKilled == True:
            preystats.prey_died()
            return True
        else:
            if self.spd <= self.slow_threshold * self.max_speed:
                gain = (self.energy_gain * dt)
                if self.energy <= (self.max_energy - gain):
                    self.energy += gain
                del gain
            else:
                self.energy -= (self.energy_drain * dt)
            
            if self.check_repr() is True:
                Preys.append(self.reproduce())  #!!!!?? speed bug?
                self.energy -= self.repr_energy_loss
                preystats.prey_repr()
            
            hits = self.raycast()
            if hits[0] < -0.8:
                preystats.prey_died()
                return True
            else:
                outp = self.NNet.evaluate(hits)
                self.spd = self.max_speed * outp[0]
                self.ang_speed = self.max_ang_speed * outp[1]
                self.look_direction += (self.ang_speed * dt)
                self.move(dt)
                del outp
                return False
    
    def raycast(self):
        hit_rays = np.zeros(self.look_lines, dtype = np.float32)
        for j in range(len(Preds)):
            other = Preds[j]
            dist = math.dist(other.location, self.location)
            if dist < (self.sight_range + other.size):
                if dist < (self.size + other.size):
                    # Collision!
                    hit_rays[0] = -0.99
                    break
                else:
                    theta_sep = self.FoV / (self.look_lines - 1)
                    theta_start = self.look_direction - (self.FoV / 2)
                    for i in range(self.look_lines):
                        CNew = RotVect(theta_start + (i * theta_sep)) @ (other.location - self.location)
                        if CNew[0] > -other.size and CNew[1] > -other.size and CNew[1] < other.size:
                            hit_rays[i] = 1
            else:
                continue
                
        return hit_rays
    
    def reproduce(self):
        child = Prey(self.location[0], self.location[1], 1.0, PreyObj = self)
        return child
    
    def killed(self):
        self.isKilled = True
    
##############################

class Predator(Being):
    max_speed = 3.5             # units / sec  
    look_lines =  7             # Number of look lines - must be equal to n1
    sight_range = 8.0           # Range of sight lines
    size = 1.0                  # Size of object
    m_extent = 4.0              # Extent of mutation
    energy_drain = 1.5          # Energy drain per second
    n2 = 12                     # HL1 size
    n3 = 6                      # HL2 size
    FoV = 75.0                  # full FoV
    repr_energy_loss = 14.0     # Energy lost when reproducing
    
    def __init__(self, x, y, scale, PredObj = None):
        if isinstance(PredObj, Predator):
            super(Predator, self).__init__(x, y, self.look_lines, self.n2, self.n3, scale, PredObj)        
        else:
            super(Predator, self).__init__(x, y, self.look_lines, self.n2, self.n3, scale)          
        
    def act(self, dt):
        if self.energy <= 0.0:
            predstats.pred_died()
            return True
        else:          
            self.energy -= (self.energy_drain * dt)
            
            if self.check_repr(True) is True:
                Preds.append(self.reproduce())
                self.energy -= self.repr_energy_loss
                predstats.pred_repr()

            hits = self.raycast()
            if hits[0] < -0.8:
                self.energy = self.max_energy
                predstats.pred_killed()

            outp = self.NNet.evaluate(hits)
            self.spd = self.max_speed * outp[0]
            self.ang_speed = self.max_ang_speed * outp[1]
            self.look_direction += (self.ang_speed * dt)
            self.move(dt)
            del outp
            return False
    
    def raycast(self):
        hit_rays = np.zeros(self.look_lines, dtype = np.float32)
        for j in range(len(Preys)):
            other = Preys[j]
            dist = math.dist(other.location, self.location)
            if dist < (self.sight_range + other.size):
                if dist < (self.size + other.size):
                    # Collision!
                    hit_rays = -np.ones(self.look_lines, dtype = np.float32)
                    other.killed()
                    break
                else:
                    theta_sep = self.FoV / (self.look_lines - 1)
                    theta_start = self.look_direction - (self.FoV / 2)
                    for i in range(self.look_lines):
                        CNew = RotVect(theta_start + (i * theta_sep)) @ (other.location - self.location)
                        if CNew[0] > -other.size and CNew[1] > -other.size and CNew[1] < other.size:
                            hit_rays[i] = 1
            else:
                continue
                
        return hit_rays
    
    def reproduce(self):
        child = Predator(self.location[0], self.location[1], 1.0, PredObj = self)
        return child

2 Options on what to simulate are:

    1) Large, simple population:
           Predator-prey simulation
        
    2) Small, complex population:
            Evolve to find food under nontrivial scenarios
            
Going with 1)

In [6]:
def make_window(dimh, dimw , totaltime, font_n, font_d, f_thickness, h_thickness, h_scale, f_scale):
    blankslate = np.zeros((dimh, dimw, 3), dtype = np.uint8)
    border = (255, 255, 0)
    
    cv.line(blankslate, (0, 0), (dimw - 1, 0), border, 1)
    cv.line(blankslate, (0, 0), (0, dimh - 1), border, 1)
    cv.line(blankslate, (dimw - 1, 0), (dimw - 1, dimh - 1), border, 1)
    cv.line(blankslate, (0, dimh - 1), (dimw - 1, dimh - 1), border, 1)

    cv.line(blankslate, (dimh - 1, 0), (dimh - 1, dimh - 1), border, 1)
    
    # Alive stats:
    cv.putText(blankslate, 'Alive', (math.floor(dimh * 1.02), math.ceil(dimh * 0.06)), font_d, h_scale, font_color, h_thickness, cv.LINE_AA)
    cv.putText(blankslate, 'Preds: ', (math.floor(dimh * 1.02), math.ceil(dimh * 0.11)), font_d, f_scale, font_color, f_thickness, cv.LINE_AA)
    cv.putText(blankslate, 'Preys: ', (math.floor(dimh * 1.02), math.ceil(dimh * 0.15)), font_d, f_scale, font_color, f_thickness, cv.LINE_AA)
    line1h = math.floor(dimh * 0.17)
    cv.line(blankslate, (dimh - 1, line1h), (dimw - 1, line1h), border, 1)
    # Write all numbers at x = dimh * 1.11
    
    # Predator stats:
    cv.putText(blankslate, 'Predators', (math.floor(dimh * 1.02), math.ceil(dimh * 0.22)), font_d, h_scale * 0.8, font_color, h_thickness, cv.LINE_AA)
    cv.putText(blankslate, 'Kills: ', (math.floor(dimh * 1.02), math.ceil(dimh * 0.27)), font_d, f_scale, font_color, f_thickness, cv.LINE_AA)
    # Write at x = dimh * 1.09 ^
    cv.putText(blankslate, 'Deaths: ', (math.floor(dimh * 1.02), math.ceil(dimh * 0.31)), font_d, f_scale, font_color, f_thickness, cv.LINE_AA)
    # Write at x = dimh * 1.12 ^ 
    cv.putText(blankslate, 'Reprs: ', (math.floor(dimh * 1.02), math.ceil(dimh * 0.35)), font_d, f_scale, font_color, f_thickness, cv.LINE_AA)
    # Write at x = dimh * 1.11 ^
    line2h = math.floor(dimh * 0.37)
    cv.line(blankslate, (dimh - 1, line2h), (dimw - 1, line2h), border, 1)
    
    # Prey stats:
    cv.putText(blankslate, 'Preys', (math.floor(dimh * 1.02), math.ceil(dimh * 0.42)), font_d, h_scale * 0.8, font_color, h_thickness, cv.LINE_AA)
    cv.putText(blankslate, 'Deaths: ', (math.floor(dimh * 1.02), math.ceil(dimh * 0.47)), font_d, f_scale, font_color, f_thickness, cv.LINE_AA)
    # Write at x = dimh * 1.12 ^
    cv.putText(blankslate, 'Reprs: ', (math.floor(dimh * 1.02), math.ceil(dimh * 0.51)), font_d, f_scale, font_color, f_thickness, cv.LINE_AA)
    # Write at x = dimh * 1.11 ^
    line3h = math.floor(dimh * 0.53)
    cv.line(blankslate, (dimh - 1, line3h), (dimw - 1, line3h), border, 1)
    
    # Sim Time elapsed:
    line4h = math.floor(dimh * 0.84)
    cv.line(blankslate, (dimh - 1, line4h), (dimw - 1, line4h), border, 1)
    cv.putText(blankslate, 'Sim Time', (math.floor(dimh * 1.02), math.ceil(dimh * 0.89)), font_d, h_scale * 0.8, font_color, h_thickness, cv.LINE_AA)
    cv.putText(blankslate, 'Elapsed: ', (math.floor(dimh * 1.02), math.ceil(dimh * 0.94)), font_d, f_scale, font_color, f_thickness, cv.LINE_AA)
    # Write at x = dimh * 1.14 with 2 decimal floats ^
    cv.putText(blankslate, 'Total: ' + str(totaltime), (math.floor(dimh * 1.02), math.ceil(dimh * 0.98)), font_n, f_scale, font_color, f_thickness, cv.LINE_AA)

    return blankslate
    

In [9]:
# SIMULATION SETTINGS:
gap = 25.0           # Smallest initial seperation between 2 beings of same type
sqrn = 5             # Number of beings of a type along x or y axis
tickrate = 20.0      # Simulation steps for every second of simulation time
sc = 0.8             # Initial scaling factor for randomized neural network
sim_time = 60.0     # Total simulation time to be passed
base_rep = 0.1       # Base reproduction chance (per second at full energy)

# Video rendering settings:
frame_rate = tickrate                       # Framerate of output video, set to tickrate for 1x playback speed
fourcc = cv.VideoWriter_fourcc(*'MJPG')     # API
imgscale = 10.0                             # Image scaling
filename = '001.avi'                        # File name
n_f = cv.FONT_HERSHEY_DUPLEX                # Font for numbers            
t_f = cv.FONT_HERSHEY_COMPLEX               # Font for text
f_t = 1                                     # Thickness of text
h_t = 2                                     # Thickness of text of headers
font_color = (255,255,255)                  # Color of font
h_sc = 1                                    # Size scaling of header text   
f_s = 0.5                                   # Size scaling of normal text

# Don't modify anything below here
BOUNDS = sqrn * gap
dim_h = math.floor(imgscale * BOUNDS)
dim_w = math.ceil(dim_h * 1.3)
dim = (dim_w, dim_h)
out = cv.VideoWriter(filename, fourcc, frame_rate, dim)
res_scaling_factor = dim_h / 650.0
f_s *= res_scaling_factor
h_sc *= res_scaling_factor
f_t = math.floor(res_scaling_factor * f_t)
h_t = math.floor(res_scaling_factor * h_t)

initial_beings = 2 * (sqrn * sqrn)
Preys = []
Preds = []
predstats = Pred_Stats()
preystats = Prey_Stats()
delta = 1.0 / tickrate
total_ticks = math.floor(sim_time * tickrate)
i_range = math.floor(sqrn * sqrn)

# Make a template for the frame:
bs = make_window(dim_h, dim_w, sim_time, n_f, t_f, f_t, h_t, h_sc, f_s)

for i in range(i_range):
    predX = (i % sqrn) * gap
    predY = (i // sqrn) * gap
    Preds.append(Predator(predX, predY, sc))

for i in range(i_range):
    preyX = ((i % sqrn) * gap) + (gap / 2)
    preyY = ((i // sqrn) * gap) + (gap / 2)
    Preys.append(Prey(preyX, preyY, sc))

print("Total Beings: ", initial_beings)
print("Bounds: ", BOUNDS)
print("Image Dimension: ", dim)

Total Beings:  50
Bounds:  125.0
Image Dimension:  (1625, 1250)


In [10]:
# MAIN SIM LOOP:
i = 0
tps = 0.0
st = time.time()

while i <= total_ticks:

    custom_img = bs.copy()  
    clear_output(wait = False) 
    
    # Simulation live statistics:
    done = ((i * 100) / total_ticks)
    print(f"Done: {done:.2f} %")
    print("Preys: ", len(Preys), "Preds: ", len(Preds))
    print("Predator K/D/R: ", predstats.kills, '/', predstats.deaths, '/', predstats.reprs)
    print("Prey ND/R: ", preystats.deaths - predstats.kills, '/', preystats.reprs)
    
    pl = len(Preys)
    pd = len(Preds)
    
    if pl * pd == 0:
        print("One type eradicated.")
        break
      
    for prey in Preys:
        ploc = (prey.location * imgscale).astype(dtype = np.uint16)
        cv.circle(custom_img , ploc, math.ceil(prey.size * imgscale), (0, 255, 0), -1)
        if prey.act(delta) == True:
            Preys.remove(prey)

    for pred in Preds:
        ploc = (pred.location * imgscale).astype(dtype = np.uint16)
        cv.circle(custom_img , ploc, math.ceil(pred.size * imgscale), (0, 0, 255), -1)
        if pred.act(delta) == True:
            Preds.remove(pred)
    
    cv.putText(custom_img, str(len(Preds)), (math.floor(dim_h * 1.11), math.ceil(dim_h * 0.11)), n_f, f_s, font_color, f_t, cv.LINE_AA)
    cv.putText(custom_img, str(len(Preys)), (math.floor(dim_h * 1.11), math.ceil(dim_h * 0.15)), n_f, f_s, font_color, f_t, cv.LINE_AA)
    cv.putText(custom_img, str(predstats.kills), (math.floor(dim_h * 1.09), math.ceil(dim_h * 0.27)), n_f, f_s, font_color, f_t, cv.LINE_AA)
    cv.putText(custom_img, str(predstats.deaths), (math.floor(dim_h * 1.13), math.ceil(dim_h * 0.31)), n_f, f_s, font_color, f_t, cv.LINE_AA)
    cv.putText(custom_img, str(predstats.reprs), (math.floor(dim_h * 1.11), math.ceil(dim_h * 0.35)), n_f, f_s, font_color, f_t, cv.LINE_AA)
    cv.putText(custom_img, str(preystats.deaths - predstats.kills), (math.floor(dim_h * 1.13), math.ceil(dim_h * 0.47)), n_f, f_s, font_color, f_t, cv.LINE_AA)
    cv.putText(custom_img, str(preystats.reprs), (math.floor(dim_h * 1.11), math.ceil(dim_h * 0.51)), n_f, f_s, font_color, f_t, cv.LINE_AA)
    elapsed = f"{(i * delta):.1f}"
    cv.putText(custom_img, elapsed, (math.floor(dim_h * 1.14), math.ceil(dim_h * 0.94)), n_f, f_s, font_color, f_t, cv.LINE_AA)
    
    out.write(custom_img)
    i += 1

out.release()
fin = time.time()
print(f"Finished sim, took {(fin - st):.1f}s")


Done: 100.00 %
Preys:  8 Preds:  5
Predator K/D/R:  6 / 36 / 16
Prey ND/R:  76 / 65
Finished sim, took 26.0s


In [ ]:
print(err)

In [ ]:
j = 0
    while j < pl:
        ploc = (Preys[j].location * imgscale).astype(dtype = np.uint16)
        cv.circle(custom_img , ploc, math.ceil(Preys[j].size * imgscale), (0, 255, 0), -1)

        if Preys[j].act(delta) == True:
            del Preys[j]
            j -= 1
            pl -= 1

        j += 1
        
    j = 0
    while j < pd:
        ploc = (Preds[j].location * imgscale).astype(dtype = np.uint16)
        cv.circle(custom_img , ploc, math.ceil(Preds[j].size * imgscale), (0, 0, 255), -1)

        if Preds[j].act(delta) == True:
            del Preds[j]
            j -= 1
            pd -= 1

        j += 1